## 4.2. 손실 함수

In [1]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [2]:
import numpy as np

def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [3]:
def cross_entropy_error(y, t):
    return -np.sum(t * np.log(y + 1e-7))

cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [4]:
from dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_size = x_train.shape[0]
batch_size = 10

batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [5]:
def cross_entropy_error(y, t):
    if y.dim == 1:
        y = y.reshape(1, y.size)
        t = t.reshape(1, t.size)
    
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

## 4.3. 수치 미분

In [6]:
def numerical_diff(f, x):
    h = 1e-50
    return (f(x + h) - f(x)) / h

def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

In [7]:
def function_2(x):
    return x[0]**2 + x[1]**2

In [8]:
## 편미분
def numerical_diff(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2 * h)
        x[idx] = tmp_val
    
    return grad



In [9]:
numerical_diff(function_2, np.array([3.0, 4.0]))

array([6., 8.])

In [10]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        
    return grad

In [11]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for _ in range(step_num):
        grad = numerical_diff(f, x)
        x -= lr * grad
        
    return x

In [12]:
gradient_descent(function_2, np.array([-3.0, 4.0]), lr=0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

### 4.4.2 신경망에서의 기울기

In [1]:
import numpy as np

from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class SimpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)
        
    def predict(self, x):
        return x @ self.W
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss
    
    
net = SimpleNet()
print(net.W)

x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

np.argmax(p)

t = np.array([0, 0, 1])
net.loss(x, t)

[[ 0.79763083 -0.64578516  1.37162525]
 [-0.64248977  0.00703123  1.2688385 ]]
[-0.0996623  -0.38114299  1.9649298 ]


0.20099156879962823

In [3]:
net = SimpleNet()

x = np.array([0.6, 0.9])
t = np.array([0, 1, 0])

def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
dW

array([[ 0.13730613, -0.35077894,  0.21347281],
       [ 0.2059592 , -0.52616842,  0.32020922]])

## 4.5. 학습 알고리즘 구현하기

In [7]:
import numpy as np
from common.functions import *
from common.gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = x @ W1 + b1
        z1 = sigmoid(a1)
        a2 = z1 @ W2 + b2
        y = softmax(a2)
        
        return y
    
    
    def loss(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
        

In [6]:
net = TwoLayerNet(784, 100, 10)

x = np.random.randn(44, 784)
y = net.predict(x)
y.shape

(44, 10)

### 4.5.2. 미니배치 학습 구현하기

In [10]:
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
net = TwoLayerNet(784, 100, 10)

train_loss_list = []

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

for iter in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grads = net.numerical_gradient(x_batch, t_batch)
    
    for key in net.params.keys():
        net.params[key] -= learning_rate * grads[key]
        
    loss = net.loss(x_batch, t_batch)
    train_loss_list.append(loss)

KeyboardInterrupt: 